In [ ]:

!git clone https://github.com/MaastrichtU-IDS/LMKBC-2023.git

In [ ]:
!git reset --hard
!git pull -f https://github.com/MaastrichtU-IDS/LMKBC-2023.git

In [ ]:
import os
os.chdir('LMKBC-2023/')

In [2]:
!pip install accelerate -U
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 15.9 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: python -m pip install --upgrade pip
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 99.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 124.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 183.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 246.8 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: python -m pip install --upgrade pip


In [1]:
!python src/run.py --mode "train_fm"




    
   python /workspace/persistent/LMKBC-2023/src/fm_model.py  --test_fn /workspace/persistent/LMKBC-2023/data/val.jsonl --template_fn res/prompts0.csv  --output_fn /workspace/persistent/LMKBC-2023/output/filled-mask-valid.jsonl --train_fn /workspace/persistent/LMKBC-2023/data/train.jsonl --train_batch_size 256 --gpu 0  --top_k 30 --threshold 0.1  --dev_fn  /workspace/persistent/LMKBC-2023/data/train_tiny.jsonl --mode "train test" --train_epoch 50 --learning_rate 5e-5 --model_load_dir bert-large-cased --model_save_dir /workspace/persistent/LMKBC-2023/bin/train_full/fill_mask/bert-large-cased --model_best_dir  /workspace/persistent/LMKBC-2023/bin/train_full/fill_mask/bert-large-cased/best_ckpt
    
    
Traceback (most recent call last):
  File "/workspace/persistent/LMKBC-2023/src/fm_model.py", line 11, in <module>
    import transformers
ModuleNotFoundError: No module named 'transformers'


In [ ]:
import torch

print(torch.cuda.is_available())

In [11]:
!git add . 
!git commit -m 'from server' 


[main 13ca210] from server
 7 files changed, 264 insertions(+), 197 deletions(-)
 create mode 100644 logging/events.out.tfevents.1690042231.jupyterlab-gpu-3-zspsf.4406.0
 create mode 100644 logging/events.out.tfevents.1690043133.jupyterlab-gpu-3-zspsf.5294.0
 create mode 100644 logging/events.out.tfevents.1690100442.jupyterlab-gpu-3-zspsf.7581.0
 rewrite main.ipynb (92%)
 create mode 100644 src/nohup.out


In [3]:
!python /workspace/persistent/LMKBC-2023/src/fm_model.py  --test_fn /workspace/persistent/LMKBC-2023/data/val.jsonl --template_fn res/prompts0.csv  --output_fn /workspace/persistent/LMKBC-2023/output/filled-mask-valid.jsonl --train_fn /workspace/persistent/LMKBC-2023/data/train.jsonl --train_batch_size 256 --gpu 0  --top_k 30 --threshold 0.1  --dev_fn  /workspace/persistent/LMKBC-2023/data/train_tiny.jsonl --mode "train test"  --train_epoch 50 --learning_rate 5e-5   --model_load_dir  /workspace/persistent/LMKBC-2023/bin/train_full/pretrain_fill_mask/bert-large-cased/best_ckpt --model_save_dir  /workspace/persistent/LMKBC-2023/bin/train_full/fill_mask/bert-large-cased --model_best_dir  /workspace/persistent/LMKBC-2023/bin/train_full/fill_mask/bert-large-cased/best_ckpt


True
{'labels': [-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, 29182, -100, -100], 'input_ids': [101, 1109, 3199, 1137, 1352, 33331, 6641, 1330, 1352, 1137, 3199, 103, 119, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1]}
{"SubjectEntityID": "Q16189274", "SubjectEntity": "Harsh Goenka", "ObjectEntitiesID": ["Q7277463"], "ObjectEntities": ["RPG Group"], "Relation": "PersonHasEmployer"}{"SubjectEntityID": "Q724465", "SubjectEntity": "Ket", "ObjectEntitiesID": ["Q159"], "ObjectEntities": ["Russia"], "Relation": "RiverBasinsCountry"}

{'labels': [-100, -100, -100, -100, -100, 31957, -100, -100, -100, -100, -100, -100, -100], 'input_ids': [101, 1109, 1583, 20325, 2075, 103, 1112, 1103, 1352, 1137, 3199, 119, 102], 'attention_mask': [1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1]}
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyT